## Import libraries

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from numpy import random as rd
import os
import json
from sklearn.decomposition import PCA
import multiprocessing, multiprocessing.pool
import matplotlib.pyplot as plt
import seaborn as sns
import time
from collections import Counter
from sklearn.manifold import TSNE
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn import tree
import miner
%matplotlib inline
plt.style.use('ggplot')

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## Create directory to save output files

In [2]:
# Path to the miner directory
proj_path = os.path.join(os.path.expanduser('~'),'Projects','miner3')
input_path = os.path.join(proj_path, 'miner_mindata')

# create name for results folder where output files will be saved
resultsFolder = "results_miner"

# create results directory
resultsDirectory = os.path.join(proj_path,resultsFolder)
if not os.path.isdir(resultsDirectory):
    os.mkdir(resultsDirectory)
# name of the folder containing the miner network
networkFolder = "results_minCorrelation_0o2_50_allFiles"
network_path = os.path.join(proj_path, networkFolder)

## Load primary data

In [3]:
# Load expression Data
expressionFile = os.path.join(input_path, "expression","IA12Zscore.csv")
id_names = os.path.join(input_path, "identifier_mappings.txt")
expressionData, conversionTable = miner.preprocess(expressionFile, id_names)

# Load mutations
common_mutations = pd.read_csv(os.path.join(input_path, 'mutations','commonMutations.csv'),index_col=0,header=0)
translocations = pd.read_csv(os.path.join(input_path, 'mutations','translocationsIA12.csv'),index_col=0,header=0)
cytogenetics = pd.read_csv(os.path.join(input_path,'mutations','cytogenetics.csv'),index_col=0,header=0)
cytogenetics = cytogenetics.loc[:,list(set(cytogenetics.columns)&set(expressionData.columns))]
common_patients_mutations_translocations = list(set(translocations.columns)&set(common_mutations.columns))
mutation_matrix = pd.concat([common_mutations.loc[:,common_patients_mutations_translocations],translocations.loc[:,common_patients_mutations_translocations]],axis=0)

#If previous results exist, use miner.read_json to load them
revisedClusters = miner.read_json(os.path.join(network_path, "coexpressionDictionary.json"))
coexpressionModules = miner.read_json(os.path.join(network_path,"coexpressionModules.json"))
regulonModules = miner.read_json(os.path.join(network_path, "regulons.json"))
mechanisticOutput = miner.read_json(os.path.join(network_path, "mechanisticOutput.json"))
regulonDf = pd.read_csv(os.path.join(network_path, "regulonDf.csv"),index_col=0,header=0)
overExpressedMembersMatrix = pd.read_csv(os.path.join(network_path, "overExpressedMembers.csv"),index_col=0,header=0)
overExpressedMembersMatrix.index = np.array(overExpressedMembersMatrix.index).astype(str)
underExpressedMembersMatrix = pd.read_csv(os.path.join(network_path, "underExpressedMembers.csv"),index_col=0,header=0)
underExpressedMembersMatrix.index = np.array(underExpressedMembersMatrix.index).astype(str)
eigengenes = pd.read_csv(os.path.join(network_path, "eigengenes.csv"),index_col=0,header=0)
eigengenes.index = np.array(underExpressedMembersMatrix.index).astype(str)

#Load table for converting gene names
id_table = pd.read_csv(id_names,sep="\t")
id_table = id_table[id_table["Source"] == "Gene Name"]
id_table.index = id_table.iloc[:,0]

completed identifier conversion.
16216 genes were converted.


## Load network

In [4]:
# Load transcriptional programs
transcriptional_programs = miner.read_json(os.path.join(network_path, 'transcriptional_programs.json'))
program_list = [transcriptional_programs[str(key)] for key in range(len(transcriptional_programs.keys()))]

In [5]:
# Load transcriptional states
transcriptional_states = miner.read_json(os.path.join(network_path, 'transcriptional_states.json'))
states_list = [transcriptional_states[str(key)] for key in range(len(transcriptional_states.keys()))]
final_state = list(set(expressionData.columns)-set(np.hstack(states_list)))
if len(final_state)>0:
    states_list.append(final_state)
states = states_list.copy()
diff_matrix_MMRF = overExpressedMembersMatrix-underExpressedMembersMatrix

## Load primary survival data

In [6]:
# load primary survival data (i.e., corresponding to expression data training set)
survivalMMRF = pd.read_csv(os.path.join(input_path,"survival","survivalIA12.csv"),index_col=0,header=0)
survivalDfMMRF = survivalMMRF.iloc[:,0:2]
survivalDfMMRF.columns = ["duration","observed"]
overExpressedMembersMatrixMMRF = overExpressedMembersMatrix
kmDfMMRF = miner.kmAnalysis(survivalDf=survivalDfMMRF,durationCol="duration",statusCol="observed")
guanSurvivalDfMMRF= miner.guanRank(kmSurvival=kmDfMMRF)

## Define patient populations

In [7]:
baseline_patients = [name for name in expressionData.columns if name.split("_")[2]=='1']
relapse_patients = [name for name in expressionData.columns if name.split("_")[2]=='2']
all_relapse_patients = [name for name in expressionData.columns if name.split("_")[2]!='1']
print('Baseline:',len(baseline_patients),'First relapse:',len(relapse_patients),'\nAny relapse:',len(all_relapse_patients))

Baseline: 779 First relapse: 56 
Any relapse: 102


In [8]:
import importlib
importlib.reload(miner)
subtypes_translocations = [
    "RNASeq_CCND1_Call",
    "RNASeq_WHSC1_Call",
    "RNASeq_MAF_Call",
    "RNASeq_MYC_Call"
]

subtypes_cytogenetics = [
    "del17",
    "amp1q"
]

patients_by_subtype = []
for subtype in subtypes_cytogenetics:
    subtype_patients = miner.getMutations(subtype,cytogenetics)
    subtype_patients = [name for name in guanSurvivalDfMMRF.index if name in subtype_patients]
    subtype_patients.reverse()
    patients_by_subtype.append(subtype_patients)

for subtype in subtypes_translocations:
    subtype_patients = miner.getMutations(subtype,translocations)
    subtype_patients = [name for name in guanSurvivalDfMMRF.index if name in subtype_patients]
    subtype_patients.reverse()
    patients_by_subtype.append(subtype_patients)
    
remainder = [name for name in guanSurvivalDfMMRF.index if name not in np.hstack(patients_by_subtype)]
remainder.reverse()

sorted_by_subtype = np.hstack([remainder,np.hstack(patients_by_subtype),relapse_patients])

## Infer nework-constrained gene activity

In [9]:
network_activity_diff = miner.networkActivity(regulon_matrix=regulonDf.copy(),
                                                 reference_matrix=diff_matrix_MMRF.copy(),
                                                 minRegulons = 2)

## Load genes of interest

In [10]:
# define genes of interest
ccnd1 = "ENSG00000110092"
noxa = "ENSG00000141682"
bak = "ENSG00000030110"

## Analyze BCL2 family genes for t(11;14) signature

In [11]:
# Generate dataframe with genes related to venetoclax
network_activity = network_activity_diff.copy()
noxa_over_bak = {'CCND1':np.array(network_activity.loc[ccnd1,:]),
                 'NOXA':np.array(network_activity.loc[noxa,:]),
                 'BAK':np.array(network_activity.loc[bak,:]),
                 'NOXA/BAK':np.array(network_activity.loc[noxa,:])-np.array(network_activity.loc[bak,:]),                 
                }
noxa_over_bak = pd.DataFrame(noxa_over_bak)
noxa_over_bak.index = network_activity.columns

# Organize data by subtypes
venetoclax_markers = noxa_over_bak.loc[np.hstack(sorted_by_subtype),:]
venetoclax_markers = venetoclax_markers.T

In [12]:
# Generate boxplot data
boxplot_data_noxa = [noxa_over_bak.loc[subtype,'NOXA'] for subtype in patients_by_subtype]
boxplot_data_bak = [noxa_over_bak.loc[subtype,'BAK'] for subtype in patients_by_subtype]
boxplot_data_nb = [noxa_over_bak.loc[subtype,'NOXA/BAK'] for subtype in patients_by_subtype]

# Define labels for subtypes in boxplot data
labels = ["Del 17","Amp 1q","t(11;14)","t(4;14)","t(14;16)","MYC"]

# Format boxplot data for plotting
# WW: error leads to attribute error that attribute convert_objects does not exist on Series.
# which is correct, because it is an attribute of DataFrame, we need to check what is wrong here
formatted_boxplot_data_noxa = miner.boxplot_figure(boxplot_data_noxa,labels)
formatted_boxplot_data_bak = miner.boxplot_figure(boxplot_data_bak,labels)
formatted_boxplot_data_nb = miner.boxplot_figure(boxplot_data_nb,labels)

AttributeError: 'Series' object has no attribute 'convert_objects'

In [13]:
boxplots = [formatted_boxplot_data_noxa,
            formatted_boxplot_data_bak,
            formatted_boxplot_data_nb
           ]

boxplots_labels = ["NOXA","BAK","NOXA / BAK"]

# Set plot style
sns.set(font_scale=1.5,style="whitegrid")
# Instantiate figure
fig = plt.figure(constrained_layout=True,figsize=(16,12))

# Set figure axes
num_plots = 4
gs = fig.add_gridspec(num_plots, 1)

for b in range(num_plots-1):
    # Fill first subplot
    fig.add_subplot(gs[b,0])
    # Plot the orbital period with horizontal boxes
    sns.boxplot(x="label", y="data", data=boxplots[b],fliersize=0)
    # Add in points to show each observation
    sns.swarmplot(x="label", y="data", data=boxplots[b],
                  size=5, color=".3", linewidth=0)
    plt.ylabel(boxplots_labels[b])
    plt.xlabel("")

ax = fig.add_subplot(gs[num_plots-1, 0])
sns.heatmap(venetoclax_markers,cmap = "Oranges",square=False)
ax.set_yticklabels(labels = venetoclax_markers.index,rotation=0)
ax.set_xticklabels(labels="",FontSize=0)
ax.tick_params(axis='both', which='both', length=0)

plt.savefig(os.path.join(resultsDirectory,"t(11;14)_BCL2_genes_figure.pdf"),bbox_inches="tight")

NameError: name 'formatted_boxplot_data_noxa' is not defined